# Improve with embedding
<img src='ex12-3.png'>

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(999999999)

In [80]:
full_data = 'hihello'
full_data = set([i for i in full_data])
full_data # Try minor change in arrangement

{'e', 'h', 'i', 'l', 'o'}

In [81]:
word_to_ix = {'e': 0, 'h': 1, 'i': 2, 'l': 3, 'o': 4}
embeds = nn.Embedding(5, 5) # 5 chars, 5 dimensions

x_data = 'hihell'

lookup_tensor = torch.tensor([word_to_ix[i] for i in x_data], dtype=torch.long)
inputs = embeds(lookup_tensor)

In [82]:
# Need to do this. Otherwise `AutoGrad` will be `Embedding` not mine
inputs = inputs.clone().detach()
inputs

tensor([[ 1.5680, -0.6124,  2.8113, -0.5173, -0.3163],
        [ 0.0268, -1.6622, -0.3501, -0.7815,  0.8334],
        [ 1.5680, -0.6124,  2.8113, -0.5173, -0.3163],
        [ 0.4023, -0.6747, -1.4315,  1.0956,  0.9050],
        [ 1.7756,  0.9900,  0.1204,  0.6238,  0.7079],
        [ 1.7756,  0.9900,  0.1204,  0.6238,  0.7079]])

In [83]:
tmp = [word_to_ix[i] for i in 'ihello']
labels = torch.tensor(tmp, dtype=torch.long)

In [84]:
labels

tensor([2, 1, 0, 3, 3, 4])

# (2) Parameters

In [85]:
num_classes = 5
input_size = 5  # one_hot size
hidden_size = 5 # output from the LSTM. 5 to directly predict one-hot
batch_size = 1  # one sentence
sequence_length = 1 # Let's do one by one
num_layers = 1  # one-layer rnn

# 1. Model

In [86]:
class Model(nn.Module):
    def __init__(self,
                input_size=5,
                hidden_size=5,
                num_layers=1,
                batch_size=1,
                sequence_length=1,
                num_classes=5):
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size,
                         hidden_size=hidden_size,
                         batch_first=True)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.num_classes = num_classes
        
        # Fully-Connected layer
        self.fc = nn.Linear(num_classes, num_classes)

    def forward(self, x, hidden):
        # Reshape input in (batch_size, sequence_length, input_size)
        x = x.view(self.batch_size, self.sequence_length, self.input_size)

        out, hidden = self.rnn(x, hidden)
        out = self.fc(out) # Add here
        out = out.view(-1, self.num_classes)
        return hidden, out
    
    def init_hidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size)


# 2. Criterion & Loss

In [87]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# 3. Training

In [88]:
model = Model(input_size=5, hidden_size=5, num_layers=1, 
              batch_size=1, sequence_length=6, num_classes=5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [89]:
hidden = model.init_hidden()
loss = 0

In [90]:
idx = 0
for epoch in range(0, 100 + 1):
    hidden.detach_()
    hidden = hidden.detach()
    hidden = hidden.clone().detach().requires_grad_(True) # New syntax from `1.0`
    
    hidden, outputs = model(inputs, hidden)
    optimizer.zero_grad()
    loss = criterion(outputs, labels) # It wraps for-loop in here
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print(f"epoch: {epoch}, loss: {loss.data}")
    print(f"Predicted string: {''.join(result_str)}")

epoch: 0, loss: 1.6794244050979614
Predicted string: hoeolh
epoch: 1, loss: 1.268625259399414
Predicted string: hohlll
epoch: 2, loss: 1.0309405326843262
Predicted string: hihlll
epoch: 3, loss: 0.8349608778953552
Predicted string: hihllo
epoch: 4, loss: 0.688709557056427
Predicted string: eihllo
epoch: 5, loss: 0.5810956358909607
Predicted string: eiello
epoch: 6, loss: 0.48993992805480957
Predicted string: eiello
epoch: 7, loss: 0.4031357765197754
Predicted string: eihllo
epoch: 8, loss: 0.32734414935112
Predicted string: eihllo
epoch: 9, loss: 0.25418564677238464
Predicted string: eihllo
epoch: 10, loss: 0.18673940002918243
Predicted string: eihllo
epoch: 11, loss: 0.13999824225902557
Predicted string: eihllo
epoch: 12, loss: 0.10653456300497055
Predicted string: eihllo
epoch: 13, loss: 0.08126533031463623
Predicted string: eihllo
epoch: 14, loss: 0.06213613227009773
Predicted string: eihllo
epoch: 15, loss: 0.047905366867780685
Predicted string: eihllo
epoch: 16, loss: 0.0375382900